In [31]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
import numpy as np
import random
df1=pd.read_csv('data.csv')
df1.head(1)

,Unnamed: 0,飞机转场记录号,到达\n日期,到达\n时刻,到达\n航班,到达类型,飞机型号,出发\n日期,出发\n时刻,出发\n航班,出发类型,上线\n机场,下线\n机场,可分配登机口,time_0
0,61,PK062,2018-01-19,-545,GN0523,D,N,2018-01-20,980,GN0256,D,NJH,ZDV,[ 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...,1525


In [32]:
df1_1=df1.sort_values(by='到达\n时刻',ascending=True)

df1_1.index=range(len(df1_1))
# print(min(df1_1['到达\n时刻']))
# print(max(df1_1['出发\n时刻']))
df1_1.head(5)

,Unnamed: 0,飞机转场记录号,到达\n日期,到达\n时刻,到达\n航班,到达类型,飞机型号,出发\n日期,出发\n时刻,出发\n航班,出发类型,上线\n机场,下线\n机场,可分配登机口,time_0
0,207,PK208,2018-01-19,-1140,NV847,I,W,2018-01-20,860,NV690,I,TZE,TGP,[ 1 2 3 4 5 22 23 24 25 26 27 58 59 60 61 ...,2000
1,61,PK062,2018-01-19,-545,GN0523,D,N,2018-01-20,980,GN0256,D,NJH,ZDV,[ 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...,1525
2,71,PK072,2018-01-19,-505,GN0497,D,N,2018-01-20,1025,GN0644,D,OOH,YNO,[ 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...,1530
3,88,PK089,2018-01-19,-450,NV663,I,W,2018-01-20,810,NV692,I,MIS,ZWS,[ 1 2 3 4 5 22 23 24 25 26 27 58 59 60 61 ...,1260
4,93,PK094,2018-01-19,-425,NV693,I,W,2018-01-20,780,NV662,I,ZWS,MIS,[ 1 2 3 4 5 22 23 24 25 26 27 58 59 60 61 ...,1205


In [33]:
def flag_or_not(gate_time,time_a,time_d):
    flag=True
    index1=(time_a-(-1140))/5
    index2=(time_d-(-1140))/5

    for i in range(int(index1),int(index2)):
        #如果有飞机停放则是不能安排新的飞机
        if gate_time[i]==0:
            flag=False
            break
    return flag
def update_time_plan(gate_time,time_a,time_d):
    index1=(time_a-(-1140))/5
    #45min 的两架飞机衔接时间
    index2=(time_d+45-(-1140))/5
    for i in range(int(index1),int(index2)):

        gate_time[i]=0
    return gate_time

In [43]:
df1_1['可分配登机口'][0]

'[ 1  2  3  4  5 22 23 24 25 26 27 58 59 60 61 62 63 64 65 66 67 68]'

In [ ]:
df3=pd.read_excel('InputData.xlsx',sheet_name='Gates')
def sort_gates(puck_gate_list,gate_list):
    dict2={}
    for choice in puck_gate_list:
        print(choice)
        dict2.update({choice:gate_list[choice]['time_plan'].count(0)/734})
    max_gates=[i[0] for i in sorted(dict2.items(), key=lambda x: x[1],reverse=True)]
    return max_gates
def generation_plan(df3=df3):
    import random
    '''
    获取登机口的数据
    Data Structure：[{区域1，终端厅1，分配的航班1，可继续分配时间1}，{区域2，终端厅2，分配的航班2，可继续分配时间2}……]
    '''

    gate_list=[]
    for i in df3.index:
        dict={'Area':df3['区域'][i],'Zhongduanting':df3['终端厅'][i],\
              'pucks_list':[],'ID':df3['登机口'][i],\
              'time_plan':[1]*734}
        gate_list.append(dict)
    '''
    获取航班的数据
    Data Structure:[{转场记录号1，到达时刻1，出发时刻1，可分配登机口1，是否被分配登机口1，登机口1，区域1，终端厅1，到达航班1，出发航班2}]
    '''
    puck_list=[]
    list1=[i for i in range(len(df1_1))]
    random.shuffle(list1)
    for i in list1:
        dict={'ID':df1_1['飞机转场记录号'][i],'Time_a':df1_1['到达\n时刻'][i],'Time_l':df1_1['出发\n时刻'][i],\
              'gates_list':df1_1['可分配登机口'][i],'flag':False,'gate':None,'Area':None,'Zhongduanting':None,\
              'puck_a':df1_1['到达\n航班'][i],'puck_l':df1_1['出发\n航班'][i],'index':i}
        puck_list.append(dict)
    '''
    首先是确定飞机和登机口的匹配规则（属性匹配、时间不冲突）
    '''
    import random
    # def generation(n,puck_list=puck_list,gate_list=gate_list):
    for puck in puck_list:

        max_gates=sort_gates(puck['gates_list'],gate_list)
        for choice in max_gates:
            # print(gate_list[choice]['pucks_list'])

            flag=flag_or_not(gate_list[choice]['time_plan'],puck['Time_a'],puck['Time_l'])
            if(flag and len(gate_list[choice]['pucks_list'])>0):
                #优先考虑现在已经利用且目前处于空闲状态的登机口--更新信息
                gate_list[choice]['time_plan']=update_time_plan(gate_list[choice]['time_plan'],puck['Time_a'],puck['Time_l'])
                gate_list[choice]['pucks_list'].append(puck['index'])
                puck['flag']=True
                puck['gate']=[choice,gate_list[choice]['ID']]
                puck['Area']=gate_list[choice]['Area']
                puck['Zhongduanting']=gate_list[choice]['Zhongduanting']
                break
            elif(flag and len(gate_list[choice]['pucks_list'])==0):
                #考虑目前为利用且目前处于空闲状态的登机口--更新信息
                gate_list[choice]['time_plan']=update_time_plan(gate_list[choice]['time_plan'],puck['Time_a'],puck['Time_l'])
                gate_list[choice]['pucks_list'].append(puck['index'])
                puck['flag']=True
                puck['gate']=[choice,gate_list[choice]['ID']]
                puck['Area']=gate_list[choice]['Area']
                puck['Zhongduanting']=gate_list[choice]['Zhongduanting']
                break

    return [puck_list,gate_list]

In [35]:
#最大化分配航班 即计算flag中为True的航班数量
def func1(pucks):
    count=0
    for puck in pucks:
        if (puck['flag']==True):
            count=count+1
    return count

#最小化使用登机口的数量 即计算目前已经使用登机口的数量
def func2(gates):
    count=0
    for gate in gates:
        if(len(gate['pucks_list'])>0):
            count=count+1
    return count

In [36]:
df2=pd.read_excel('InputData.xlsx',sheet_name='Tickets')
df2=df2[(df2['到达\n日期']=='2018-01-20') | (df2['出发\n日期']=='2018-01-20')]
df2.head(5)

,旅客\n记录号,乘客数,到达\n航班,到达\n日期,出发\n航班,出发\n日期
1316,T1317,2,GN0641,2018-01-19,NV664,2018-01-20
1355,T1356,2,NV673,2018-01-19,NV664,2018-01-20
1356,T1357,2,NV673,2018-01-19,NV664,2018-01-20
1357,T1358,2,NV673,2018-01-19,NV664,2018-01-20
1358,T1359,2,NV673,2018-01-19,NV664,2018-01-20


In [37]:
str(df2['到达\n日期'][1360])[:10]

'2018-01-19'

In [38]:
tickets=[]
for i in df2.index:
    #确定进场航班的index
    df2_1=df1_1[(df1_1['到达\n航班']==df2.loc[i]['到达\n航班'])&(df1_1['到达\n日期']==str(df2.loc[i]['到达\n日期'])[:10])]
    if len(df2_1.index)>0:
        index1=df2_1.index[0]
    else:
        index1=-99
    #确定出场航班的index
    df2_2=df1_1[(df1_1['出发\n航班']==df2.loc[i]['出发\n航班'])&(df1_1['出发\n日期']==str(df2.loc[i]['出发\n日期'])[:10])]
    if len(df2_2.index)>0:
        index2=df2_2.index[0]
    else:
        index2=-99

    dict={'ID':df2.loc[i]['旅客\n记录号'],'count':df2.loc[i]['乘客数'],'index_a':index1,'index_l':index2,\
          'puck_a':df2.loc[i]['到达\n航班'],'puck_l':df2.loc[i]['出发\n航班'],'time_a':str(df2.loc[i]['到达\n日期'])[:10],\
          'time_l':str(df2.loc[i]['出发\n日期'])[:10]}
    tickets.append(dict)

In [41]:
[puck_list,gate_list]=generation_plan()

[


TypeError: list indices must be integers or slices, not str

In [ ]:
'''
函数功能：根据乘客的信息，计算中转时间（航站楼的行走时间 乘坐捷运时间）
进而确定出适合每一架飞机起飞的登机口的序列
'''
puck_list[tickets[0]['index_l']][]









